# User-defined metadata

Next to system metadata, iRODS allows you to create own metadata with data objects and collections.

You can use that metadata to describe your data and later search for this data; and it can help you keeping the overview of what was the input for an analysis and what is the outcome.

<img src="img/DataObject5.png" width="400">

Technically, iRODS offers metadata as key-value-units triple. Let's investigate this:

## Add metadata to data objects

As always: first we have to create an iRODS session:

In [ ]:
from ibridges.interactive import interactive_auth
session = interactive_auth()

Make sure we have our demo collection and file available:

In [ ]:
from ibridges.path import IrodsPath

irods_path = IrodsPath(session, '~')
print("Current working location:", irods_path)
irods_coll_path = irods_path.joinpath('demo')
print("New collection name:", irods_coll_path)
coll = IrodsPath.create_collection(session, irods_coll_path)
print("New collection is created:", irods_coll_path.collection_exists())

Now we can retrieve a data object and insect its metadata.

In [ ]:
from ibridges.path import IrodsPath

irods_coll_path = IrodsPath(session, '~').joinpath('demo')
obj = irods_coll_path.joinpath('demofile.txt')

print(obj.meta)

Most probably you will see no metadata in the above cell. **Note, that system metadata and user-defined metadata are two different entities in a data object!**
With the command `MetaData(obj)` we only retrieve the user-defined metadata.

<img src="img/DataObject4.png" width="400">

Now we can add some own metadata. The metadata comes as key-value-units triple:

In [ ]:
obj.meta.add('Key', 'Value', 'Units')
print(obj.meta)

Sometimes we do not really have `units`, so we can leave this part empty:

In [ ]:
obj.meta.add('Author', 'Christine')
print(obj.meta)

We can also add a second author:

In [ ]:
obj.meta.add('Author', 'Raoul')
print(obj.meta)

You see, that keys in **iRODS metadata keys can have different values**. That is different from python dictionaries where one key can only have one value. **How then to overwrite a value?**

## Overwrite metadata

If you wish to *overwrite* a value, you can first add the new metadata key-value-units triple as above and subsequently remove the old one. **NOTE, that all entries with the key will be deleted.** If you want to be more specific you will need to give the value and the units.

In [ ]:
obj.meta.delete('Author')
print(obj.meta)

In [ ]:
obj.meta.add('Author', 'Raoul')
obj.meta.add('Author', 'Christine')
print(obj.meta)

In [ ]:
obj.meta.delete('Author', 'Christine')
print(obj.meta)

You can also set all existing values to **one** new value:

In [ ]:
obj.meta.set('Author', 'Maarten')
print(obj.meta)

## Add metadata to collections

The same functionality we saw above, we can use for collections:

In [ ]:
coll = irods_coll_path
print(coll.meta)

In [ ]:
coll.meta.add('TypeOfCollection', 'Results')
print(coll.meta)

## Which metadata can help you keeping an overview?

iRODS metadata can help you keeping an overview while you are working with data and maybe many files which have relations to each other. There are ontologies which define keywords and links between keywords like the **[prov-o Ontology](https://www.w3.org/TR/prov-o/#prov-o-at-a-glance)**.

Let's see how we can annotate our test data, so that we know that it is test data.

In [ ]:
from datetime import datetime
coll.meta.add('prov:wasGeneratedBy', 'Christine')
coll.meta.add('CollectionType', 'testcollection')
obj.meta.add('prov:SoftwareAgent', 'iRODS jupyter Tutorial')
obj.meta.add('prov:wasGeneratedBy', 'Maarten')
obj.meta.add('DataType', 'testdata')

Now we have some more descriptive metadata that gives us hints, in which context the data was created:

In [ ]:
print(coll.meta)
print(obj.meta)

## Finding data by their metadata

Metadata does not only help you to. keep an overview over your data, but can also be used to select and retrieve data. In iBridges you can use the user-defined metadata and some system metadata fields to search for data.

In our first example, we are looking for objects and collections called *demo* in our `home`:

In [ ]:
from ibridges.search import search_data, MetaSearch
result = search_data(session, path=session.home, path_pattern="demo")
print(result)

If no `path` is provided, *ibridges* will automatically fall back on your `home`.

The output is a list of IrodsPath's indicating the locations of the data objects and collections.

In [ ]:
result = search_data(session, metadata=MetaSearch(key='prov:wasGeneratedBy', value='Christine'))
print(result)

If we do not want to specify the particular value for this metadata entry, we can leave it out.

In [ ]:
result = search_data(session, metadata=MetaSearch(key='prov:wasGeneratedBy'))
print(result)

Now we also receive the data object that was generated by *Maarten*.

And of course we can combine information about the path and the metadata. they will be connected with `and`. The following search will retrieve all data objects and collections wich are labeled with a metadata key *'prov:wasGeneratedBy'* and whose path has the prefix */nluu12p/home/research-test-christine/demo/*.

In [ ]:
result = search_data(session, path=IrodsPath(session, session.home, 'demo'),
                     metadata=MetaSearch(key='prov:wasGeneratedBy'))
print(result)

## Retrieving data

Now that we have the search results we can use the `IrodsPath` to download them or to fetch more information:

In [ ]:
print(result[0].size)
print(result[0].collection_exists())
print(result[0].dataobject_exists())

# Metadata archives

In most cases the user is encouraged to access and manipulate metadata through the `MetaData` class. However, there are some cases where it can be useful to create an archive of all metadata in a collection and all subcollections and data objects. One example might be a backup of the data and metadata on a system that does not support metadata. Another might be to easily transfer metadata from one iRODS system to another. A final use case might be having access to the metadata during computation on a system that is not connected to the internet.

## Creating a metadata archive

In [ ]:
from ibridges.data_operations import create_meta_archive

collection_path = IrodsPath(session, "~", "demo")
create_meta_archive(session, collection_path, "meta_archive.json")

## Applying a metadata archive

This restores/overwrites the metadata on the iRODS server with the metadata from the archive. Make sure that the paths of the subcollections and data objects have not changed.

In [ ]:
from ibridges.data_operations import apply_meta_archive

apply_meta_archive(session, "meta_archive.json", collection_path)

This creates a file in your current local directory of this jupyter notebook.

In [ ]:
!cat meta_archive.json